#### Import Abstracts

# Move to small dataset and see how many questions it takes!!!!!!

In [1]:
import pandas as pd
data = pd.read_csv("Abstracts-186.csv")
data = data.drop('Unnamed: 0', axis=1)
data

,Entity,Label,Abstract
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A..."
...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...


In [2]:
data['Triples'][0]

"[['is', '2004 action gothic horror film written'], ['is', '2004 action horror film written by Stephen Sommers'], ['is', 'gothic written'], ['is', '2004 action horror film'], ['is', '2004 action gothic horror film'], ['is', '2004 action horror film written'], ['is', '2004 action gothic horror film written by Stephen Sommers'], ['stars Hugh Jackman as', 'monster hunter Van Helsing'], ['stars Hugh Jackman as', 'Dutch monster hunter Van Helsing'], ['stars', 'Hugh Jackman'], ['is homage to', 'Universal Horror Monster films from 1930s'], ['is', 'homage produced by Universal Pictures'], ['is', 'homage to Universal Horror Monster films produced'], ['is', 'homage also produced by Universal Pictures'], ['is', 'homage to Universal Horror Monster films from 1930s also produced by Universal Pictures'], ['is', 'homage'], ['is', 'homage to Universal Horror Monster films from 1930s produced by Universal Pictures'], ['is', 'homage to Universal Horror Monster films from 1930s also produced'], ['is', 'h

#### Model

##### Whole code (Too slow to run)

##### Extract triples (Around 3 hours)

In [2]:
from openie import StanfordOpenIE
from transformers import pipeline
import spacy

properties = {
    'openie.affinity_probability_cap': 1 / 3,
}

unmasker = pipeline('fill-mask', model='bert-base-uncased')
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrip = []
j = 0
first_name = 0
with StanfordOpenIE(properties=properties) as client:
    
    for s,label,abstract in data.values:

        if type(abstract) is float:
            datatrip.append([])
            print(j,label, 'nothing')
            j += 1
            continue

        print(j,label)


        #text = abstract
        #print('Text: %s.' % text)
        for triple in client.annotate(abstract):
            if first_name == 0:
                name = triple['subject']
                first_name += 1
            #print('|-', triple)
            if any(triple['subject'].casefold() == item for item in item_list) or triple['subject'] in name or triple['subject'] in label:
                triples.append(triple)
        first_name = 0
            
        datatrip.append(triples)
        triples = []
        j += 1

/Users/ab/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 David Bowie
Starting server with command: java -Xmx8G -cp /Users/ab/.stanfordnlp_resources/stanford-corenlp-4.5.3/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-34bc9ed9b2e645ff.props -preload openie
1 Kanye West
2 Eminem
3 Jennifer Aniston
4 George W. Bush
5 Prince (musician)
6 Justin Bieber
7 Drake (musician)
8 Heath Ledger
9 John Krasinski
10 Val Kilmer
11 American Horror Story
12 Brad Pitt
13 Japan
14 Barack Obama
15 Bill Clinton
16 Donald Trump
17 Jimmy Carter
18 Martin Luther King Jr.
19 Ronald Reagan
20 Germany
21 Matthew Perry
22 Vietnam War
23 The Walking Dead (TV series)
24 London
25 World War I
26 California
27 The Beatles
28 Robert Downey Jr.
29 Roger Federer
30 Marilyn Manson
31 Ben Affleck
32 India
33 Serena Williams
34 Novak Djokovic
35 China
36 France
37 Michael Jackson
38 Nicolas Cage
39 Johnny Depp
40 South Africa
41 Tobey Maguire
42 Sandra Bullock
43 Paul Walk

In [3]:
data['Stanford-Trips-Raw'] = datatrip
data
# if save needed
#data.to_csv('Stanfordpart1.csv')

,Entity,Label,Abstract,Stanford-Trips-Raw
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...,"[{'subject': 'Jones', 'relation': 'was', 'obje..."
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...,"[{'subject': 'Ye', 'relation': 'is', 'object':..."
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1...","[{'subject': 'He', 'relation': 'is credited wi..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196...","[{'subject': 'Aniston', 'relation': 'is', 'obj..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A...","[{'subject': 'Bush', 'relation': 'is', 'object..."
...,...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...,"[{'subject': 'Pelosi', 'relation': 'is', 'obje..."
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te...","[{'subject': 'AP', 'relation': 'is', 'object':..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...,"[{'subject': 'MH', 'relation': 'is state in', ..."
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...,[{'subject': 'Paris Saint Germain Football Clu...


In [4]:
testing = []
done = []
j = 0
for s,label,abstracts,triples in data.values:
    #print(j, label)
    if triples is None:
        done.append([])
    for triple in triples:
        testing.append([triple['relation'], triple['object']])
    
    done.append(testing)
    testing = []
    j += 1

In [5]:
data['Stanford-Trips-Raw'] = done
data
#if save needed
#data.to_csv('Stanfordpart1.csv')

,Entity,Label,Abstract,Stanford-Trips-Raw
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...,"[[was, English], [was acclaimed by, critics pa..."
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...,"[[is, American], [is, regarded], [is, widely r..."
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1...","[[is credited with, hip hop], [is credited wit..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196...","[[is, American], [is recipient of, accolades i..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A...","[[is, American], [served as, governor of Texas..."
...,...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...,"[[is, American], [has represented, California ..."
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te...","[[is, state], [is state in, south eastern coas..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...,"[[is state in, western region], [is, state in ..."
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...,"[[referred to, to Paris Saint Germain], [commo..."


In [6]:
maxi = 0
for s,l,a,t in data.values:
    maxi = max(maxi,len(t))
    if maxi == len(t):
        print(l,len(t))

David Bowie 133
Eminem 157
Drake (musician) 266
President of the United States 763


In [7]:
#import pandas as pd
#data = pd.read_csv("Stanfordpart1.csv")
#data = data.drop('Unnamed: 0', axis=1)
#data

##### Process Triples (Too long to run)

In [ ]:
import ast
from transformers import pipeline
import spacy

unmasker = pipeline('fill-mask', model='bert-base-uncased', top_k=1, framework='pt',device=-1)
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrips = []
j = 0
trips = []
history = []
for s,label,abstract,triples in data.values[j:]:
    
    if type(abstract) is float:
            datatrips.append([])
            print(j,label, 'nothing')
            j += 1
            continue
            
    print(j,label)
    for triple in triples:
    #for triple in ast.literal_eval(triples):
        # turn triple into string 
        string = (label,triple[0],triple[1])
        string = (' '.join(string))
        tokens = nlp(string)

        for chunk in tokens.noun_chunks:
            chunks.append(chunk.root.head.text)

        # if something in chunk
        if len(chunks) > 0:

            if chunks[-1] == 'is' or chunks[-1] == 'was':
                ob = nlp(triple[1])

                # get noun chunks from object
                for b in ob.noun_chunks:
                    # make triple
                    if ([chunks[-1],b.text]) not in history:
                        history.append([chunks[-1],b.text])
                        slm = ("%s. %s %s [MASK] %s" %(string, label, chunks[-1], b.text))
                        print(slm) 
                        maskrel = unmasker(slm)[0]['token_str']
                        print([maskrel, b.text])
                        print()
                        trips.append([maskrel, b.text])

            else:
                ob = nlp(triple[1])

                # get noun chunks from object
                for b in ob.noun_chunks:
                    # make triple
                    slm = ("%s. %s [MASK] %s %s" %(string, label, chunks[-1], b.text))
                    print(slm)
                    if ([chunks[-1],b.text]) not in history:
                        history.append([chunks[-1],b.text])
                        mask = unmasker(slm)[0]['token_str']
                    # if MASK is punctuation, remove MASK
                        if mask in punc:
                            print([chunks[-1], b.text])
                            print()
                            trips.append([chunks[-1], b.text])

                        else:
                            print(["%s%s" %(mask, chunks[-1].capitalize()),b.text])
                            print()
                            trips.append(["%s%s" %(mask, chunks[-1].capitalize()),b.text])

        else:
            trips.append(['%s' %(triple[0]),'%s' %(triple[1])])
            # empty chunk list
        chunks = []
    datatrips.append(trips)
    trips = []
    history = []
    j += 1
    print()
    print()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 David Bowie
David Bowie was English. David Bowie was [MASK] English
['not', 'English']

David Bowie was acclaimed by critics particularly for his innovative work during 1970s. David Bowie [MASK] for critics
['for', 'critics']

David Bowie was acclaimed by critics particularly for his innovative work during 1970s. David Bowie [MASK] for his innovative work
['particularlyFor', 'his innovative work']

David Bowie was acclaimed by critics. David Bowie [MASK] by critics
['acclaimedBy', 'critics']

David Bowie was acclaimed by critics for his work. David Bowie [MASK] for critics
David Bowie was acclaimed by critics for his work. David Bowie [MASK] for his work
['awardFor', 'his work']

David Bowie was acclaimed by critics for his innovative work. David Bowie [MASK] for critics
David Bowie was acclaimed by critics for his innovative work. David Bowie [MASK] for his innovative work
David Bowie was acclaimed by critics particularly for his work. David Bowie [MASK] for critics
David Bowie was 

small - 30 mins

# If program breaks:

In [8]:
datatrips1 = datatrips
j1 = j

In [9]:
import ast
from transformers import pipeline
import spacy

unmasker = pipeline('fill-mask', model='bert-base-uncased', top_k=1, framework='pt',device=-1)
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
#datatrips1 = []
#j1 = 0
trips = []
history = []
for s,label,abstract,triples in data.values[j1:]:
    
    if type(abstract) is float:
            datatrips1.append([])
            print(j1,label, 'nothing')
            j1 += 1
            continue
            
    print(j1,label)
    #for triple in triples:
    for triple in ast.literal_eval(triples):
        # turn triple into string 
        string = (label,triple[0],triple[1])
        string = (' '.join(string))
        tokens = nlp(string)

        for chunk in tokens.noun_chunks:
            chunks.append(chunk.root.head.text)

        # if something in chunk
        if len(chunks) > 0:

            if chunks[-1] == 'is' or chunks[-1] == 'was':
                ob = nlp(triple[1])

                # get noun chunks from object
                for b in ob.noun_chunks:
                    # make triple
                    if ([chunks[-1],b.text]) not in history:
                      #print('ye - 1',[chunks[-1],b.text])
                        history.append([chunks[-1],b.text])
                        slm = ("%s. %s %s [MASK] %s" %(string, label, chunks[-1], b.text))
                    # check not already in triples
                    #if ([unmasker(slm)[0]['token_str'], b.text]) not in trips:
                        trips.append([unmasker(slm)[0]['token_str'], b.text])

            else:
                ob = nlp(triple[1])

                # get noun chunks from object
                for b in ob.noun_chunks:
                    # make triple
                    slm = ("%s. %s [MASK] %s %s" %(string, label, chunks[-1], b.text))
                    if ([chunks[-1],b.text]) not in history:
                      #print('ye-',[chunks[-1],b.text])
                        history.append([chunks[-1],b.text])
                        mask = unmasker(slm)[0]['token_str']
                    # if MASK is punctuation, remove MASK
                        if mask in punc:
                          #if ([chunks[-1], b.text]) not in trips:
                              # check not already in triples
                            trips.append([chunks[-1], b.text])

                        else:
                          #if (["%s%s" %(unmasker(slm)[0]['token_str'], chunks[-1].capitalize()),b.text]) not in trips:
                              # check not already in triples
                            trips.append(["%s%s" %(mask, chunks[-1].capitalize()),b.text])

        else:
            trips.append(['%s' %(triple[0]),'%s' %(triple[1])])
            # empty chunk list
        chunks = []
    #print(trips)
    datatrips1.append(trips)
    trips = []
    history = []
    j1 += 1

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


4268 Yellow fever
4269 Frankie Faison
4270 Alison Krauss
4271 Jack Nicholson
4272 Falkirk F.C.
4273 Ava Gardner
4274 Japanese Americans
4275 Bob Dylan
4276 Brian Robbins
4277 Cyrano de Bergerac (1990 film)
4278 MGM Records
4279 Michael Redgrave
4280 Burt Reynolds
4281 Desperate Housewives
4282 Stuart Craig
4283 Ruth Prawer Jhabvala
4284 Joseph Barbera
4285 Joel Edgerton
4286 FC Mariupol
4287 Heartbeats (film) nothing
4288 Chester City F.C.
4289 North Dakota
4290 Victor%2FVictoria nothing
4291 Tom Petty and the Heartbreakers
4292 Ernest Lehman
4293 Corsica
4294 The West Wing
4295 John Williams
4296 Dante Ferretti
4297 Masako Nozawa
4298 Tinnu Anand
4299 Trentino
4300 Sinhalese people
4301 Al Jourgensen
4302 Rosario Dawson
4303 Summer of Sam
4304 Wikimedia Foundation
4305 Susan Blu
4306 Acclaim Entertainment
4307 Chatham County, Georgia
4308 Warren County, Ohio
4309 Alexander Korda
4310 DePaul University
4311 MTV Movie Award for Best On-Screen Duo
4312 Cruel Intentions
4313 Chemist
4314 

-28th 18:47:46 



- 16:38:29 : 4268

In [33]:
data['Stanford'] = datatrips
len(datatrips)
data
# if save needed
#data.to_csv('Stanfordpart2.csv')

,Entity,Label,Abstract,Stanford-Trips-Raw,Stanford
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...,"[[was, English], [was acclaimed by, critics pa...","[[not, English], [for, critics], [particularly..."
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...,"[[is, American], [is, regarded], [is, widely r...","[[workedFor, producer], [producedFor, Roc A-Fe..."
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1...","[[is credited with, hip hop], [is credited wit...","[[creditedWith, hip hop], [specializesIn, hip ..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196...","[[is, American], [is recipient of, accolades i...","[[awardsIncluding, accolades], [awardsIncludin..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A...","[[is, American], [served as, governor of Texas...","[[sonOf, governor], [universityOf, Texas], [se..."
...,...,...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...,"[[is, American], [has represented, California ...","[[livesIn, California 's district], [servedIn,..."
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te...","[[is, state], [is state in, south eastern coas...","[[a, state], [partOf, south eastern coastal re..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...,"[[is state in, western region], [is, state in ...","[[stateIn, western region], [isIn, state], [is..."
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...,"[[referred to, to Paris Saint Germain], [commo...","[[referredTo, Paris Saint Germain], [competeIn..."


In [12]:
#import pandas as pd
#data = pd.read_csv('Stanfordpart2.csv')
#data = data.drop('Unnamed: 0', axis=1)
#data

In [34]:
num = 0
for i in data.Stanford:
    num += len(i)
num
    

11553

In [35]:
final_trips = []
import nltk
import ast
trips = []
k = 0
for s,label,abstract,_,triples in data.values:
    print(k)
    k+=1
    if type(abstract) == float:
        print('ye')
        final_trips.append([])
        continue
    
    #for triple in ast.literal_eval(triples):
    for triple in triples:
        if triple[0] == 'is' or triple[0] == 'also' or triple[0] == 'an' or triple[0] == 'a' or triple[0] == 'was' or triple[0] == 'as' or triple[0] == 'not' or triple[0] == 'the':
            simpletok = str(triple[1]).split(' ')
            if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also':
                print(triple[0],'- type'," ".join(simpletok[1:]), '-', label)
                if len(" ".join(simpletok[1:])) == 0:
                       print(label, triple[0], '- Removed' )
                else:
                       trips.append(['type'," ".join(simpletok[1:])])
            else:
                print(triple[0], '-', 'type', triple[1], '-', label)
                trips.append(['type', triple[1]])
        else:
            simpletok = str(triple[1]).split(' ')
            if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also':
                print(" ".join(simpletok[1:]), '-', label)
                if len(" ".join(simpletok[1:])) == 0:
                       print(label, triple[0], '- Removed' )
                else:
                       trips.append([triple[0], " ".join(simpletok[1:])])
            else:
                trips.append(triple)
    final_trips.append(trips)
    trips = []

0
not - type English - David Bowie
as - type professional career - David Bowie
as - type musician - David Bowie
as - type career - David Bowie
a - type vinyl artist - David Bowie
1
also - type founder - Kanye West
2
a - type member - Eminem
a - type music artist - Eminem
3
the - type recipient - Jennifer Aniston
career - Jennifer Aniston
career - Jennifer Aniston
role - Jennifer Aniston
role - Jennifer Aniston
role - Jennifer Aniston
as - type one performers - Jennifer Aniston
the - type co - Jennifer Aniston
4
5
a - type singer songwriter - Prince (musician)
an - type American singer songwriter - Prince (musician)
a - type prolific musician - Prince (musician)
a - type musician - Prince (musician)
6
 - Justin Bieber
Justin Bieber number - Removed
 - Justin Bieber
Justin Bieber Justice - Removed
7
also - type Certified Lover Boy - Drake (musician)
8
an - type Australian actor - Heath Ledger
an - type actor - Heath Ledger
as - type his role - Heath Ledger
9
also - type recipient - John 

In [36]:
import re
trips = []
fin = []
for triples in final_trips:
    for triple in triples:
        pred = re.sub( r"([A-Z])", r" \1", triple[0]).split()
        if len(pred) > 1:
            if pred[0] == 'also' or pred[0] == 'was' or pred[0] == 'also' or pred[0] == 'has':
                relation = pred[1].lower()
                simpletok = str(triple[1]).split(' ')
                if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also' or simpletok[0] == 'his' or simpletok[0] == 'she':
                    if len(" ".join(simpletok[1:])) == 0:
                        continue
                        #print(triple[0], triple[1], '- Removed' )
                        #print()
                    else:
                        #print('fix')
                        #print(triple)
                        #print(relation," ".join(simpletok[1:]))
                        #print()
                        trips.append([relation," ".join(simpletok[1:])])
                else:
                    trips.append([relation,triple[1]])
            else:
                trips.append(triple)
        else:
            trips.append(triple)
    fin.append(trips)
    trips = []
fin

[[['type', 'English'],
  ['for', 'critics'],
  ['particularlyFor', 'his innovative work'],
  ['acclaimedBy', 'critics'],
  ['awardFor', 'his work'],
  ['developed', 'interest'],
  ['grewFrom', 'interest'],
  ['retiredFrom', 'music'],
  ['musicFrom', 'early age'],
  ['developedFrom', 'age'],
  ['developedIn', 'interest'],
  ['interestedIn', 'music'],
  ['type', 'professional career'],
  ['type', 'musician'],
  ['embarkedOn', 'career'],
  ['embarkedOn', 'professional career'],
  ['type', 'career'],
  ['studied', 'art'],
  ['emergedDuring', 'glam rock era'],
  ['diedDuring', 'rock era'],
  ['emergedAfter', 'period'],
  ['endOf', 'period'],
  ['periodOf', 'experimentation'],
  ['of', 'single Starman'],
  ['characterisedAs', 'soul'],
  ['has', 'style'],
  ['characterisedAs', 'plastic soul'],
  ['starredIn', 'cult film'],
  ['againWith', 'album Low'],
  ['changedWith', 'album'],
  ['changed', 'direction'],
  ['continuedWith', 'three collaborations'],
  ['changedWith', 'inflected album'],
  [

In [17]:
trips = []
done = []
nouns = []
import spacy 
nlp = spacy.load("en_core_web_sm")

for triples in fin: 
    for triple in triples:
        dd = triple[0].split(' ')
        if len(dd) > 1:
      #print(triple[0], '-', label, triple[0].split(' ')[1], triple[1])
            tokens = nlp(str(triple[1]))
            for token in tokens.noun_chunks:
                nouns.append(token)
            if len(nouns) == 1:
                print(triple[0], '-', label, triple[0].split(' ')[1], nouns[0])
                trips.append([triple[0].split(' ')[1], nouns[0]])
            else:
                tokenized = nltk.word_tokenize(str(triple[1]))
                pos = nltk.pos_tag(tokenized)
                for i in pos:
                    if i[1] == 'CD' or i[1] == 'NN':
                        print(triple[0], '-', label, triple[0].split(' ')[1], i[0])
                        trips.append([triple[0].split(' ')[1], i[0]])
            nouns = []
        else:
            trips.append(triple)
    done.append(trips)
    trips = []

Together own - Uttar Pradesh own about 14 %


In [37]:
trips = []
final = []
for triples in fin:
    for triple in triples:
        obj = str(triple[1]).split(" ")
        obj = "_".join(obj)
        trips.append([triple[0], obj])
    final.append(trips)
    trips = []
len(final), final[:5]

(184,
 [[['type', 'English'],
   ['for', 'critics'],
   ['particularlyFor', 'his_innovative_work'],
   ['acclaimedBy', 'critics'],
   ['awardFor', 'his_work'],
   ['developed', 'interest'],
   ['grewFrom', 'interest'],
   ['retiredFrom', 'music'],
   ['musicFrom', 'early_age'],
   ['developedFrom', 'age'],
   ['developedIn', 'interest'],
   ['interestedIn', 'music'],
   ['type', 'professional_career'],
   ['type', 'musician'],
   ['embarkedOn', 'career'],
   ['embarkedOn', 'professional_career'],
   ['type', 'career'],
   ['studied', 'art'],
   ['emergedDuring', 'glam_rock_era'],
   ['diedDuring', 'rock_era'],
   ['emergedAfter', 'period'],
   ['endOf', 'period'],
   ['periodOf', 'experimentation'],
   ['of', 'single_Starman'],
   ['characterisedAs', 'soul'],
   ['has', 'style'],
   ['characterisedAs', 'plastic_soul'],
   ['starredIn', 'cult_film'],
   ['againWith', 'album_Low'],
   ['changedWith', 'album'],
   ['changed', 'direction'],
   ['continuedWith', 'three_collaborations'],
   

In [38]:
data['Stanford'] = final
data

,Entity,Label,Abstract,Stanford-Trips-Raw,Stanford
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...,"[[was, English], [was acclaimed by, critics pa...","[[type, English], [for, critics], [particularl..."
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...,"[[is, American], [is, regarded], [is, widely r...","[[workedFor, producer], [producedFor, Roc_A-Fe..."
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1...","[[is credited with, hip hop], [is credited wit...","[[creditedWith, hip_hop], [specializesIn, hip_..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196...","[[is, American], [is recipient of, accolades i...","[[awardsIncluding, accolades], [awardsIncludin..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A...","[[is, American], [served as, governor of Texas...","[[sonOf, governor], [universityOf, Texas], [se..."
...,...,...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...,"[[is, American], [has represented, California ...","[[livesIn, California_'s_district], [servedIn,..."
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te...","[[is, state], [is state in, south eastern coas...","[[type, state], [partOf, south_eastern_coastal..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...,"[[is state in, western region], [is, state in ...","[[stateIn, western_region], [isIn, state], [is..."
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...,"[[referred to, to Paris Saint Germain], [commo...","[[referredTo, Paris_Saint_Germain], [competeIn..."


In [39]:
data.to_csv('Stanford-186.csv')

In [13]:
pip install rfc3987

  Using cached rfc3987-1.3.8-py2.py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.


In [40]:
import rdflib
from rdflib import Graph
from rdflib.namespace import RDF
from rdflib import Literal, XSD
from rfc3987 import parse
flag = True
g = Graph()
k = 0
n = 0
for s,label,abstract,_,triples in data.values:
    
    if type(abstract) == float:
        continue
        
    for triple in triples:
        flag = True
        
        try:
            int(triple[1])
        except:
            flag = False 
    
        if flag is True:
            rel = triple[0].replace("'", '')
            rel = rel.replace('"', '')
            rel = rel.replace('`', '')
            rel = rel.replace('.', '')
            schema = ('http://schema.org/%s' %(rel))
            year = int(triple[1])
            year = Literal(year, datatype=XSD.gYear)
            try:
                parse(schema, rule='IRI')
            except:
                n+=1
                print(n,'bad -',triple)
                continue
            g.add( (rdflib.URIRef(s), rdflib.URIRef(schema), year ))
            k+=1
        else:
            obj = triple[1].replace("'", '')
            obj = obj.replace('"', '')
            obj = obj.replace('`', '')
            obj = obj.replace('.', '')
            link = 'http://yago-knowledge.org/resource/%s' %(obj)
            if triple[0] == 'type':
                try:
                    parse(link, rule='IRI')
                except:
                    n+=1
                    print(link)
                    print(n,'bad -',triple)
                    continue
                g.add( (rdflib.URIRef(s), RDF.type, rdflib.URIRef(link)) )
                k+=1
            else:
                rel = triple[0].replace("'", '')
                rel = rel.replace('"', '')
                rel = rel.replace('`', '')
                rel = rel.replace('.', '')
                schema = ('http://schema.org/%s' %(rel))
                try:
                    parse(link, rule='IRI')
                except:
                    n+=1
                    print(n,'bad -',triple)
                    continue
                try:
                    parse(schema, rule='IRI')
                except:
                    n+=1
                    print(n,'bad -',triple)
                    continue
                g.add( (rdflib.URIRef(s), rdflib.URIRef(schema), rdflib.URIRef(link)) )
                k+=1

1 bad - ['accountingOf', '13.4_%']
2 bad - ['accountingFor', '13.4_%']
3 bad - ['accountsFor', 'only_1.5_%']
4 bad - ['accountsOf', 'only_1.5_%']
5 bad - ['consistsOf', 'only_about_50_%']
6 bad - ['countriesWith', 'only_about_50_%']
7 bad - ['landExceeding', 'only_about_50_%']
8 bad - ['outOf', '48.6_%']
9 bad - ['himselfReceived', '48.6_%']
10 bad - ['up', '48.6_%']
11 bad - ['marginOf', '55.9_%']
12 bad - ['canOwn', 'about_14_%']
13 bad - ['Together own', 'about_14_%']
14 bad - ['shareOf', 'about_14_%']
15 bad - ['comesWith', '75_%_market_share']
16 bad - ['There became', 'involved']
17 bad - ['##heringTo', 'Leninist_interpretation']
18 bad - ['accountingFor', 'at_least_70_%']
19 bad - ['accountingOf', 'at_least_70_%']
20 bad - ['accountsOf', 'just_over_4.2_%']
21 bad - ['hostedOver', '30_%_growth']
22 bad - ['stateHosted', '30_%_growth']
23 bad - ['broughtIn', '30_%_growth']
24 bad - ['consistsOf', '7.3_%']
25 bad - ['equalTo', '7.3_%']


In [41]:
total = 0
for triples in data['Stanford']:
    if len(triples) == 0:
        continue 
    else:
        for triple in triples:
            total += 1
total,k

(11495, 11470)

In [1]:
n,total, n/total

NameError: name 'n' is not defined

(25, 11495, 0.002174858634188778)

Big - 793/374742 triples were removed for not fitting the proper format, thats 0.21%

small - 25/11494 triples were removed for not fitting the proper format, thats 0.22%

In [43]:
g.serialize(destination='Stanford-186.ttl', format='turtle')

<Graph identifier=N20ee95fe3da14307be8838384a2ab344 (<class 'rdflib.graph.Graph'>)>